In [15]:
import json
import sys
sys.path.append('..')

import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM

from utils.prompt_utils import TransformData


# Load data and transform it to X, y (X being a prompt)

In [8]:
data = json.load(open("cosmosqa_10k.json"))

In [9]:
data_transformer = TransformData(data)
X, y = data_transformer.transform_data()

# Load the model and its tokenizer

In [13]:
def load_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto", cache_dir="/mnt/mapie_llm/cache")
    model.eval()
    return tokenizer, model

# %%
tok, mo = load_model("mistralai/Mistral-7B-Instruct-v0.3")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Wrap the model

In [16]:
class LLMClassifier:

    
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model
        self.options = ["Answer: A", "Answer: B", "Answer: C", "Answer: D"]
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.option_ids = [self.tokenizer.encode(opt)[-1] for opt in self.options]
        

    def predict_proba(self, prompts):
        if isinstance(prompts, str):
            prompts = [prompts]  # Convert single prompt to list
        pred_probas = []
        for prompt in prompts:
            prompt = [prompt]
            inputs = self._prepare_inputs(prompt)
            with torch.no_grad():
                outputs = self.model(**inputs)
            
            logits = outputs.logits[:, -1, :]  # Extract logits for the last token [batch_size, vocab_size]
            logits_options = logits[:, self.option_ids]  # Select logits corresponding to answer options
            
            probabilities = F.softmax(logits_options, dim=-1).cpu().numpy()
            pred_probas.append(probabilities)
        return probabilities

    def predict(self, prompts):
        probas = self.predict_proba(prompts)
        return np.argmax(probas, axis=-1)

    def _prepare_inputs(self, prompts):
        inputs = self.tokenizer(prompts, return_tensors="pt", truncation=True)
        for k in inputs:
            if torch.is_tensor(inputs[k]):
                inputs[k] = inputs[k].to("cuda")
        return inputs

In [17]:
llm = LLMClassifier(tok, mo)

In [19]:
llm.predict_proba(X[0]), llm.predict(X[0])

(array([[1.664e-03, 9.927e-01, 4.971e-03, 9.193e-04]], dtype=float16),
 array([1]))

In [20]:
y[0]

np.int64(1)